In [19]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import csv
torch.set_printoptions(edgeitems=2, linewidth=75)

In [71]:
#First we are importing the dataset

wine_path = "../data/p1ch4/tabular-wine/winequality-white.csv"
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=";",
                         skiprows=1)

wineq = torch.from_numpy(wineq_numpy)

#Splitting scores from actual input
input_data = wineq[:, :-1] #
wine_score = wineq[:, -1].long() #

#normalazing input data
input_data_mean = torch.mean(data, dim=0)
input_data_var = torch.var(data, dim=0)
input_data_normalized = (data - data_mean) / torch.sqrt(data_var)

#splitting into train and val dataset
n_samples = data.shape[0]
n_val = int(0.2 * n_samples) #int is important)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

input_data_train_norm = input_data_normalized[train_indices]
wine_score_train = wine_score[train_indices]

input_data_val_norm = input_data_normalized[val_indices]
wine_score_val = wine_score[val_indices]

print(f'Input train shape{input_data_train_norm.shape}, input validation {input_data_val_norm.shape}')

#creating one_hot representation of our score

wine_score_train_onehot = torch.zeros(wine_score_train.shape[0], 10)
wine_score_val_onehot =  torch.zeros(wine_score_val.shape[0], 10)
wine_score_train_onehot.scatter_(1, wine_score_train.unsqueeze(1), 1.0)
wine_score_val_onehot.scatter_(1, wine_score_val.unsqueeze(1), 1.0)

print(f'Score onehot train shape{wine_score_train_onehot.shape}, Score onehot validation shape{wine_score_val_onehot.shape}')

Input train shapetorch.Size([3919, 11]), input validation torch.Size([979, 11])
Score onehot train shapetorch.Size([3919, 10]), Score onehot validation shapetorch.Size([979, 10])


In [74]:
def training_loop(n_epochs, optimizer, model, loss_fn, input_data_train_norm, input_data_val_norm,
                  wine_score_train, wine_score_val):
    for epoch in range(1, n_epochs + 1):
        prediction_train = model(input_data_train_norm) # <1>
        loss_train = loss_fn(prediction_train, wine_score_train)

        prediction_val = model(input_data_val_norm) # <1>
        loss_val = loss_fn(prediction_val, wine_score_val)
        
        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")

neuron_count = 100

seq_model = nn.Sequential(
    nn.Linear(11, neuron_count),
    nn.Tanh(),
    nn.Linear(neuron_count, 10))

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(seq_model.parameters())

training_loop(
    n_epochs = 3000, 
    optimizer = optimizer,
    model = seq_model,
    loss_fn = loss_fn,
    input_data_train_norm = input_data_train_norm,
    input_data_val_norm = input_data_val_norm, 
    wine_score_train = wine_score_train_onehot,
    wine_score_val = wine_score_val_onehot)

Epoch 1, Training loss 2.2654, Validation loss 2.2555
Epoch 1000, Training loss 0.8641, Validation loss 0.9909
Epoch 2000, Training loss 0.6084, Validation loss 1.0962
Epoch 3000, Training loss 0.4536, Validation loss 1.3623


In [78]:
seq_model(data_train_norm)[2], target_train_onehot[2]

(tensor([-5.3939, -5.1573, -5.0202, -3.7603, -2.3114,  2.6902,  6.2457,
          0.5261, -6.0117, -2.4114], grad_fn=<SelectBackward0>),
 tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]))

In [39]:
neuron_count = 100

seq_model = nn.Sequential(
    nn.Linear(11, neuron_count),
    nn.Tanh(),
    nn.Linear(neuron_count, 10))


optimizer = optim.SGD(seq_model.parameters(), lr=1e-4)

# Assuming loss_fn is a loss function
loss_fn = nn.MSELoss()
optimizer = optim.Adam()
for epoch in range(1000):
    optimizer.zero_grad()
    t_p_train = seq_model(data_train_norm)
    # Convert target_train to the same data type as t_p_train
    target_train = target_train.float()
    loss_train = loss_fn(t_p_train, target_train)
    loss_train.backward()
    optimizer.step()


TypeError: Adam.__init__() missing 1 required positional argument: 'params'